## Creating the training dataset from the equilibria dataset

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1" # disable GPU
import numpy as np
from time import time, sleep
from os.path import join, exists
from utils import *
from tqdm import tqdm
print("Preparing data...")

In [ ]:
# hyperparameters
DTYPE = 'float32'
RAW_DIR = 'raw_ds' if LOCAL else '/nfsd/automatica/grandinmat/raw_ds' # where the raw data is stored

# hyperparameters
N_SAMPLES = 1000 if LOCAL else 200_000 #850_000 # number of samples to use for training
SM = 1 if LOCAL else 8 # 20 number of grids per samples (SM = SAMPLE MULTIPLIER)
TRAIN_EVAL_SPLIT = 0.8 # percentage of the dataset to use for training

print(f'Total samples: {N_SAMPLES*SM:.0f}, train samples: {N_SAMPLES*SM*TRAIN_EVAL_SPLIT:.0f}, eval samples: {N_SAMPLES*SM*(1-TRAIN_EVAL_SPLIT):.0f}')

In [ ]:
# check RAW_DIR exists and is not empty
assert exists(RAW_DIR), f'RAW_DIR {RAW_DIR} does not exist'
assert len(os.listdir(RAW_DIR)) > 0, f'RAW_DIR {RAW_DIR} is empty'

In [ ]:
# load the data
print("Loading data...")

# list all the files inside RAW_DIR/ds
files = sorted([f for f in os.listdir(RAW_DIR) if f.endswith('.mat')])
print(f'Found {len(files)} files.')
vals = {n:[] for n in DS_NAMES}  # dictionary to hold the numpy arrays
files_iter = tqdm(files, desc="Loading files", unit="file") if LOCAL else files
# files_iter = files
for f in files_iter:
    # try:
    if True:
        d = loadmat(join(RAW_DIR, f))
        # print(f'file: {f}, keys: {d.keys()}') #  'Bm0', 'Bm1', 'Ff0', 'Ff1', 'Ft0', 'Ft1', 'Fx', 'Ia0', 'Ia1', 'Ip0', 'Ip1', 'Iu0', 'Iu1', 'Iy', 'rBt0', 'rBt1', 'rq', 't', 'zq'

        for n in DS_NAMES: assert n in list(d.keys()), f'Key {n} not found in {f}' 

        t = d['t'].flatten()  # time
        nt = t.shape[0]  # number of time points

        for n in DS_NAMES:
            v = d[n] # get the variable
            v = v.reshape((*DS_SIZES[n], nt)) # try to reshape it to the expected shape
            assert v.shape == (*DS_SIZES[n], nt), f'Variable {n} shape mismatch: {v.shape} != {((*DS_SIZES[n], nt))}'
            assert not np.isnan(v).any(), f'Variable {n} contains NaN values in {f}'
            assert np.isfinite(v).all(), f'Variable {n} contains infinite values in {f}'
            # # transpose first dimension to the last
            # v = v.transpose(*range(1, len(v.shape)), 0)
            # v = v.reshape((nt, *DS_SIZES[n]))  # reshape to (nt, *DS_SIZES[n])
            vals[n].append(v)  # append to the list
    # except Exception as e:
    #     print(f'Error loading {f}: {e}')
    #     continue

print(f'Loaded {len(vals[FX])} files.')
assert len(vals[FX]) > 0, f'No samples: {len(vals[FX])}'

# convert lists to numpy arrays
print('---------------------------------------------------')
for n in DS_NAMES:
    vals[n] = np.concatenate(vals[n], axis=-1).astype(DTYPE)  # concatenate along the first dimension and convert to DTYPE
    sh, ndim = vals[n].shape, vals[n].ndim
    vals[n] = vals[n].transpose(ndim-1, *range(ndim-1))  # move the last dimension to the first
    print(f'{n} -> shape: {vals[n].shape}, dtype: {vals[n].dtype}')  # print the shape and dtype of the variable
print('---------------------------------------------------')

for n in INPUT_NAMES: assert vals[n].ndim == 2, f'Variable {n} is not 2D: {vals[n].ndim}'

# assign values to variables
X = np.concatenate([vals[n] for n in INPUT_NAMES], axis=1)  # concatenate along the first dimension
print(f'-----------------------------------------\n X -> shape: {X.shape}, dtype: {X.dtype}')

# make IY the same shape as FX (and pad the sides)
tmpIy = np.zeros_like(vals[FX])  # create a zero array with the same shape as FX
tmpIy[:, 1:-1, 1:-1] = vals[IY]  # fill the inner part with IY values
# fill sides 
tmpIy[:, 0, 1:-1] = vals[IY][:, 0, :]  # left side
tmpIy[:, -1, 1:-1] = vals[IY][:, -1, :]  # right side
tmpIy[:, 1:-1, 0] = vals[IY][:, :, 0]  # top side
tmpIy[:, 1:-1, -1] = vals[IY][:, :, -1]  # bottom side
# corners
tmpIy[:, 0, 0] = vals[IY][:, 0, 0]  # top left corner
tmpIy[:, 0, -1] = vals[IY][:, 0, -1]  # top right corner
tmpIy[:, -1, 0] = vals[IY][:, -1, 0]  # bottom left corner
tmpIy[:, -1, -1] = vals[IY][:, -1, -1]  # bottom right corner
vals[IY] = tmpIy  # replace IY with the new array

Y1 = vals[FX]  
Y2 = vals[IY]  
Y3 = np.concatenate([vals[RQ], vals[ZQ]], axis=1)  # rzq
print(f'Y1 -> shape: {Y1.shape}, dtype: {Y1.dtype}')
print(f'Y2 -> shape: {Y2.shape}, dtype: {Y2.dtype}')
print(f'Y3 -> shape: {Y3.shape}, dtype: {Y3.dtype}')

assert Y1.shape[0] > 0, f'No samples: {Y1.shape}'
N_OR = Y1.shape[0]  # number of original samples
print(f'Loaded {N_OR} samples.')

assert X.shape == (N_OR, NIN), f'X shape mismatch: {X.shape} != ({N_OR}, {NIN})'

# check the shapes
print(f'X shape: {X.shape}, Y1 shape: {Y1.shape}, Y2 shape: {Y2.shape}, Y3 shape: {Y3.shape}')

# print sizes in MB
print(f'X size: {X.nbytes / 1024**2:.2f} MB, Y1 size: {Y1.nbytes / 1024**2:.2f} MB, Y2 size: {Y2.nbytes / 1024**2:.2f} MB, Y3 size: {Y3.nbytes / 1024**2:.2f} MB')

In [ ]:
# plot some examples
n_plot = 3 if LOCAL else 15
rand_idxs = np.random.randint(0, N_OR, n_plot)
for i, ri in enumerate(rand_idxs):
    plt.figure(figsize=(16, 3))
    plt.subplot(1, 5, 1)
    # plt.contourf(RRD, ZZD, Y1[ri], levels=20)
    plt.scatter(RRD, ZZD, c=Y1[ri], s=4)
    # plot th LCFS
    plt.plot(vals[RQ][ri], vals[ZQ][ri], 'gray', lw=2)
    plot_vessel()
    plt.axis('equal'), plt.axis('off')
    plt.title('vals[Fx]')
    plt.colorbar()
    plt.subplot(1, 5, 2)
    # plt.contourf(RRD, ZZD, Y2[ri], levels=20)
    plt.scatter(RRD, ZZD, c=Y2[ri], s=4)
    plt.plot(vals[RQ][ri], vals[ZQ][ri], 'gray', lw=2)
    plot_vessel()
    plt.axis('equal'), plt.axis('off')
    plt.title('Iy')
    plt.colorbar()
    plt.subplot(1, 5, 3)
    plt.bar(np.arange(vals[IA].shape[1]), vals[IA][ri])
    plt.title('vals[IA]')
    plt.subplot(1, 5, 4)
    plt.bar(np.arange(vals[BM].shape[1]), vals[BM][ri])
    plt.title('vals[BM]')
    plt.subplot(1, 5, 5)
    plt.bar(np.arange(vals[FF].shape[1]), vals[FF][ri])
    plt.title('vals[FF]')
    plt.tight_layout()
    plt.suptitle(f'SHOT {ri}')
    plt.show() if LOCAL else plt.savefig(f'{DS_DIR}/imgs/original_{i}.png')
    plt.close()

In [ ]:
# test interpolation
idx = np.random.randint(0, N_OR)
f, rhs = Y1[idx,:,:], Y2[idx,:,:]
rrg, zzg = sample_random_subgrid(RRD,ZZD, NGZ, NGR)
print(f.shape, rhs.shape, rrg.shape, zzg.shape)
box = grid2box(rrg, zzg)
f_grid = interp_fun(vals[FX][idx,:,:], RRD, ZZD, rrg, zzg)
rhs_grid = interp_fun(rhs, RRD, ZZD, rrg, zzg)

fig,ax = plt.subplots(1,5, figsize=(20,5))
ax[0].scatter(RRD, ZZD, marker='.')
ax[0].scatter(rrg, zzg, marker='.')
plot_vessel(ax[0])
ax[0].set_aspect('equal')

# im1 = ax[1].contourf(RRD, ZZD, f, 20)
im1 = ax[1].scatter(RRD, ZZD, c=f.flatten(), s=4)
ax[1].plot(box[:,0],box[:,1])
plot_vessel(ax[1])
ax[1].set_aspect('equal')

# im2 = ax[2].contourf(rrg, zzg, f_grid, 20)
im2 = ax[2].scatter(rrg, zzg, c=f_grid.flatten(), s=4)
plot_vessel(ax[2])
ax[2].set_aspect('equal')

# im3 = ax[3].contourf(RRD, ZZD, rhs, 20)
im3 = ax[3].scatter(RRD, ZZD, c=rhs.flatten(), s=4)
ax[3].set_aspect('equal')
ax[3].plot(box[:,0],box[:,1])
plot_vessel(ax[3])

# im4 = ax[4].contourf(rrg, zzg, rhs_grid, 20)
im4 = ax[4].scatter(rrg, zzg, c=rhs_grid.flatten(), s=4)
plot_vessel(ax[4])
ax[4].set_aspect('equal')

plt.colorbar(im1,ax=ax[1])
plt.colorbar(im2,ax=ax[2])
plt.colorbar(im3,ax=ax[3])
plt.colorbar(im4,ax=ax[4])

plt.show() if LOCAL else plt.savefig(join(DS_DIR, 'imgs', 'interpolation_example.png'))
plt.close()

In [ ]:
# dataset splitting (N_TOP = original dataset size)
NT = int(N_SAMPLES*TRAIN_EVAL_SPLIT)    # training
NE = N_SAMPLES - NT                     # evaluation
NTM, NEM = NT*SM, NE*SM # training and evaluation with multiple grids
print(f"Train -> NT:{NT} NTM:{NTM}")
print(f"Eval  -> NE:{NE} NEM:{NEM}")
orig_idxs = np.random.permutation(N_OR)
orig_idxs_train = orig_idxs[:int(N_OR*TRAIN_EVAL_SPLIT)] # original indices for training
orig_idxs_eval = orig_idxs[int(N_OR*TRAIN_EVAL_SPLIT):] # original indices for evaluation
# splitting the idxs
assert len(orig_idxs_train) > NT, f"Training set is too small, {len(orig_idxs_train)} < {NT}"
idxs_t = np.random.choice(orig_idxs_train, NT, replace=False) # can overlap with idxs_tf
assert len(orig_idxs_eval) > NE, f"Evaluation set is too small, {len(orig_idxs_eval)} < {NE}"
idxs_e = np.random.choice(orig_idxs_eval, NE, replace=False) # can overlap with idxs_ef

In [ ]:
# create arrays to store the dataset
print(f"Preallocating arrays for the dataset...")

xt =   np.zeros((NTM, NIN), dtype=DTYPE)
y1t =  np.zeros((NTM, NGZ, NGR), dtype=DTYPE)
y2t =  np.zeros((NTM, NGZ, NGR), dtype=DTYPE)
y3t =  np.zeros((NTM, 2*NLCFS), dtype=DTYPE)
rt =   np.zeros((NTM, NGR), dtype=DTYPE)
zt =   np.zeros((NTM, NGZ), dtype=DTYPE)

xe =   np.zeros((NEM, NIN), dtype=DTYPE)
y1e =  np.zeros((NEM, NGZ, NGR), dtype=DTYPE)
y2e =  np.zeros((NEM, NGZ, NGR), dtype=DTYPE)
y3e =  np.zeros((NEM, 2*NLCFS), dtype=DTYPE)
re =   np.zeros((NEM, NGR), dtype=DTYPE)
ze =   np.zeros((NEM, NGZ), dtype=DTYPE)

# estimate RAM usage
ram_usage = sum(arr.nbytes for arr in [xt, y1t, y2t, y3t, rt, zt, xe, y1e, y2e, y3e, re, ze]) / 1024**3 
print(f"Estimated RAM usage: {ram_usage:.2f} GB\nFilling arrays...")

## fill the arrays
print_every = 2000
start_time = time()
for i, idx in enumerate(idxs_t):
    rrs, zzs = np.zeros((SM,NGZ,NGR), dtype=DTYPE), np.zeros((SM,NGZ,NGR), dtype=DTYPE)
    for j in range(SM): rrs[j], zzs[j] = sample_random_subgrid(RRD,ZZD,NGZ,NGR)
    si, ei = i*SM, (i+1)*SM # start and end idxs
    xt[si:ei] = X[idx]
    y1t[si:ei] = interp_fun(Y1[idx], RRD, ZZD, rrs, zzs)
    y2t[si:ei] = interp_fun(Y2[idx], RRD, ZZD, rrs, zzs)
    y3t[si:ei] = Y3[idx]
    rt[si:ei], zt[si:ei] = rrs[:,0,:], zzs[:,:,0] # save only the first raw/col
    if (i+1) % print_every == 0: print(f"Train -> {100*(i+1)*SM/NTM:.1f}%, eta: {((time()-start_time)/(i+1)*(NT-i-1))/60:.1f} min")

start_time = time()
for i, idx in enumerate(idxs_e):
    rrs, zzs = np.zeros((SM,NGZ,NGR), dtype=DTYPE), np.zeros((SM,NGZ,NGR), dtype=DTYPE)
    for j in range(SM): rrs[j], zzs[j] = sample_random_subgrid(RRD,ZZD,NGZ,NGR)
    si, ei = i*SM, (i+1)*SM # start and end idxs
    xe[si:ei] = X[idx]
    y1e[si:ei] = interp_fun(Y1[idx], RRD, ZZD, rrs, zzs)
    y2e[si:ei] = interp_fun(Y2[idx], RRD, ZZD, rrs, zzs)
    y3e[si:ei] = Y3[idx]
    re[si:ei], ze[si:ei] = rrs[:,0,:], zzs[:,:,0] # save only the first raw/col
    if (i+1) % print_every == 0: print(f"Eval -> {100*(i+1)*SM/NEM:.1f}%, eta: {((time()-start_time)/(i+1)*(NE-i-1))/60:.1f} min")

print(f"xt: {xt.shape}, y1t: {y1t.shape}, y2t: {y2t.shape}, y3t: {y3t.shape}, rt: {rt.shape}, zt: {zt.shape}")
print(f"xe: {xe.shape}, y1e: {y1e.shape}, y2e: {y2e.shape}, y3e: {y3e.shape}, re: {re.shape}, ze: {ze.shape}")

In [ ]:
# calculate kernels for Grad-Shafranov equation # NOTE: not needed actually, but here to be tested
# so we don't have to do it during training
print("Calculating kernels...")
laplace_ker_t = np.zeros((len(xt[0]), 3, 3), dtype=DTYPE)
laplace_ker_e = np.zeros((len(xe[0]), 3, 3), dtype=DTYPE)
df_dr_ker_t = np.zeros((len(xt[0]), 3, 3), dtype=DTYPE)
df_dr_ker_e = np.zeros((len(xe[0]), 3, 3), dtype=DTYPE)
# hrs_t, hzs_t = rt[:,1,2]-rt[:,1,1], zt[:,2,1]-zt[:,1,1]
# hrs_e, hzs_e = re[:,1,2]-re[:,1,1], ze[:,2,1]-ze[:,1,1]
hrs_e, hzs_e = re[:,2]-re[:,1], ze[:,2]-ze[:,1]
hrs_t, hzs_t = rt[:,2]-rt[:,1], zt[:,2]-zt[:,1]
for i in range(len(xt[0])):
    try:
        laplace_ker_t[i,:,:], df_dr_ker_t[i,:,:] = calc_laplace_df_dr_ker(hrs_t[i], hzs_t[i])
    except Exception as e:
        print(f"Error calculating laplace_ker_t for index {i}: {e}")
        plt.figure()
        plt.scatter(rt[i], zt[i], marker='.')
        plt.title(f"rt[{i}]")
        plt.axis('equal')
        plt.show() if LOCAL else plt.savefig(f'{DS_DIR}/imgs/rr_train_{i}.png')
        plt.close()
        break

for i in range(len(xe[0])):
    laplace_ker_e[i,:,:], df_dr_ker_e[i,:,:] = calc_laplace_df_dr_ker(hrs_e[i], hzs_e[i])

In [ ]:
# check the dataset
print("Checking the dataset...")
rows = 5
idxs_train = np.random.randint(0, len(xt[0]), rows)
idxs_eval = np.random.randint(0, len(xe[0]), rows)
fig,ax = plt.subplots(rows,6, figsize=(20,4*rows))
box0 = grid2box(RRD, ZZD)
for i, (it, ie)  in enumerate(zip(idxs_train, idxs_eval)):
    # training
    boxi = grid2box(rt[it], zt[it])
    ax[i,0].plot(box0[:,0], box0[:,1])
    ax[i,0].plot(boxi[:,0], boxi[:,1])
    ax[i,0].set_aspect('equal')
    ax[i,0].set_title(f"Train {it}")
    a1 = ax[i,1].contourf(rt[it], zt[it], y1t[it], 20)
    ax[i,1].plot(box0[:,0], box0[:,1])
    ax[i,1].plot(y3t[it,:NLCFS], y3t[it,NLCFS:], 'gray', lw=2)
    ax[i,1].set_aspect('equal')
    plt.colorbar(a1,ax=ax[i,1])
    a2 = ax[i,2].contourf(rt[it], zt[it], y2t[it], 20)
    ax[i,2].plot(box0[:,0], box0[:,1])
    ax[i,2].set_aspect('equal')
    plt.colorbar(a2,ax=ax[i,2])
    # evaluation
    boxi = grid2box(re[ie], ze[ie])
    ax[i,3].plot(box0[:,0], box0[:,1])
    ax[i,3].plot(boxi[:,0], boxi[:,1])
    ax[i,3].set_aspect('equal')
    ax[i,3].set_title(f"Eval {ie}")
    a1 = ax[i,4].contourf(re[ie], ze[ie], y1e[ie], 20)
    ax[i,4].plot(box0[:,0], box0[:,1])
    ax[i,4].plot(y3e[ie,:NLCFS], y3e[ie,NLCFS:], 'gray', lw=2)
    ax[i,4].set_aspect('equal')
    plt.colorbar(a1,ax=ax[i,4])
    a2 = ax[i,5].contourf(re[ie], ze[ie], y2e[ie], 20)
    ax[i,5].plot(box0[:,0], box0[:,1])
    ax[i,5].set_aspect('equal')
    plt.colorbar(a2,ax=ax[i,5])
plt.show() if LOCAL else plt.savefig(join(DS_DIR, 'imgs', 'dataset_check.png'))
plt.close()

In [ ]:
from utils import calc_gso, calc_gso_batch
import torch
print("Checking the Grad-Shafranov operator...")
n_plots = 7
idxs = np.random.randint(0, len(xt[0]), n_plots)
psis, rhss = y1t[idxs], y2t[idxs]
rs, zs = rt[idxs], zt[idxs]
big_box = grid2box(RRD, ZZD)
#batched version
psist = torch.tensor(psis, dtype=torch.float32).view(n_plots, 1, NGZ, NGR)
rst = torch.tensor(rs, dtype=torch.float32).view(n_plots, NGR)
zst = torch.tensor(zs, dtype=torch.float32).view(n_plots, NGZ)
print(f'psi: {psist.shape}, r: {rst.shape}, z: {zst.shape}')
gsos = calc_gso_batch(psist, rst, zst)
print(f'gsos: {gsos.shape}')
gsos = gsos.view(n_plots, NGZ, NGR).numpy()
# single version
for i in range(n_plots):
    psi, r, z, rhs = psis[i], rs[i], zs[i], rhss[i]
    box = grid2box(r, z)
    gso = calc_gso(psi, r, z) # calculate the Grad-Shafranov operator
    gso2 = gsos[i]
    #plot error gso vs gso2
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    rr, zz = np.meshgrid(r, z)
    im = ax.contourf(rr, zz, np.abs(gso-gso2), 20)
    ax.plot(big_box[:,0], big_box[:,1])
    ax.set_aspect('equal')
    ax.set_title(f"Error batch/no batch {i}")
    plt.colorbar(im, ax=ax)
    plt.show() if LOCAL else plt.savefig(join(DS_DIR, 'imgs', f'gso_error_{i}.png'))
    plt.close()
    # NOTE: the error between the batched and non-batched version can be non-zero due to different
    # implementations in gpu
    print(f"max error batch/no batch: {np.abs(gso-gso2).max()}")
    # assert np.allclose(gso, gso2, rtol=1e-2), f"Error in the calculation of the Grad-Shafranov operator: \ngso:\n{gso}, \ngso2:\n{gso2}"
    # psi, gso, rhs = psi[1:-1,1:-1], gso[1:-1,1:-1], rhs[1:-1,1:-1]
    # rr, zz = rr[1:-1,1:-1], zz[1:-1,1:-1] 
    fig,ax = plt.subplots(1,5, figsize=(20,5))
    ax[0].plot(big_box[:,0], big_box[:,1])
    ax[0].plot(box[:,0], box[:,1])
    ax[0].set_aspect('equal')
    ax[0].set_xticks([]), ax[0].set_yticks([])
    ax[0].set_title(f"Train {idxs}")
    im1 = ax[1].contourf(rr, zz, psi, 20)
    ax[1].plot(big_box[:,0], big_box[:,1])
    ax[1].set_aspect('equal')
    ax[1].set_xticks([]), ax[1].set_yticks([])
    ax[1].set_title("Ψ")
    im2 = ax[2].contourf(rr, zz, -gso, 20)
    ax[2].plot(big_box[:,0], big_box[:,1])
    ax[2].set_aspect('equal')
    ax[2].set_xticks([]), ax[2].set_yticks([])
    ax[2].set_title("GSO recalculated")
    im3 = ax[3].contourf(rr, zz, -rhs, 20)
    ax[3].plot(big_box[:,0], big_box[:,1])
    ax[3].set_aspect('equal')
    ax[3].set_xticks([]), ax[3].set_yticks([])
    ax[3].set_title("GSO from dataset")
    im4 = ax[4].contourf(rr, zz, np.abs(gso-rhs), 20)
    ax[4].plot(big_box[:,0], big_box[:,1])
    ax[4].set_aspect('equal')
    ax[4].set_xticks([]), ax[4].set_yticks([])
    ax[4].set_title("Absolute error")
    plt.colorbar(im1,ax=ax[1])
    plt.colorbar(im2,ax=ax[2])
    plt.colorbar(im3,ax=ax[3])
    plt.colorbar(im4,ax=ax[4])
    plt.show() if LOCAL else plt.savefig(join(DS_DIR, 'imgs', f'gso_check_{i}.png'))
    plt.close()

In [ ]:
assert y1t.shape[1:] == (NGR, NGZ), f"xt shape mismatch: {xt[0].shape[1:]} != ({NGR}, {NGZ})"
assert y1e.shape[1:] == (NGR, NGZ), f"xe shape mismatch: {xe[0].shape[1:]} != ({NGR}, {NGZ})"

In [ ]:
# load dataset and normalize, NOTE: y3 does not require normalization
print("Normalizing the dataset...")
μx, Σx = np.mean(xt, axis=0), np.std(xt, axis=0)
# μy1, Σy1 = np.mean(y1t, axis=0), np.std(y1t, axis=0)
# μy2, Σy2 = np.mean(y2t, axis=0), np.std(y2t, axis=0)
μy1, Σy1 = np.mean(y1t), np.std(y1t)
μy2, Σy2 = np.mean(y2t), np.std(y2t)

# normalize (NOTE: both with the training means and stds)
# xt, xe = (xt - μx) / Σx, (xe - μx) / Σx # not normalizing the inputs bc we added a normalization layer in the network architecture
y1t, y1e = (y1t - μy1) / Σy1, (y1e - μy1) / Σy1
y2t, y2e = (y2t - μy2) / Σy2, (y2e - μy2) / Σy2

print(f'μx: {μx.shape}, Σx: {Σx.shape}')
print(f'μy1: {μy1.shape}, Σy1: {Σy1.shape}')
print(f'μy2: {μy2.shape}, Σy2: {Σy2.shape}')

x_mean_std = np.array([μx, Σx])
y1_mean_std = np.array([μy1, Σy1])
y2_mean_std = np.array([μy2, Σy2])

print(f'x_mean_std: {x_mean_std.shape}')
print(f'y1_mean_std: {y1_mean_std.shape}')  
print(f'y2_mean_std: {y2_mean_std.shape}')

In [ ]:
# save dataset as numpy compressed
print(f"Saving datasets to {TRAIN_DS_PATH} and {EVAL_DS_PATH}...")
try:
    np.savez_compressed(TRAIN_DS_PATH, X=xt, Y1=y1t, Y2=y2t, Y3=y3t, r=rt, z=zt, x_mean_std=x_mean_std, y1_mean_std=y1_mean_std, y2_mean_std=y2_mean_std)
    np.savez_compressed(EVAL_DS_PATH, X=xe, Y1=y1e, Y2=y2e, Y3=y3e, r=re, z=ze, x_mean_std=x_mean_std, y1_mean_std=y1_mean_std, y2_mean_std=y2_mean_std)
except Exception as e:
    print(f"Error saving datasets: {e}")
    raise e
print(f"Datasets saved.")

In [ ]:
# save a .mat demo file for testing
print(f"Saving demo file to {DS_DIR}/demo.mat and {DS_DIR}/demo.npz")
rand_idxs = np.random.randint(0, len(xe), 100)
demo = {'X': xe[rand_idxs], 'Y': y3e[rand_idxs]}
try:
    savemat(join(DS_DIR, 'demo.mat'), demo)
    np.savez_compressed(join(DS_DIR, 'demo.npz'), X=xe[rand_idxs], Y=y3e[rand_idxs])
except Exception as e:
    print(f"Error saving demo file: {e}")
    raise e

In [ ]:
# plot some examples
print("Testing the dataset...")
tds, eds = np.load(TRAIN_DS_PATH), np.load(EVAL_DS_PATH)
print(f'train_ds: {tds.keys()}')
print(f'eval_ds: {eds.keys()}')
# plot some examples
rs, zs, xs, y1s, y2s, y3s = tds['r'], tds['z'], tds['X'], tds['Y1'], tds['Y2'], tds['Y3']
xs = (xs - μx) / Σx # normalize the inputs
print(f'rs shape: {rs.shape}, zs shape: {zs.shape}, xs shape: {xs.shape}, y1s shape: {y1s.shape}, y2s shape: {y2s.shape}, y3s shape: {y3s.shape}')

n_plot = 3 if LOCAL else 100
rand_idxs = np.random.randint(0, NT, n_plot)
for i, ri in enumerate(rand_idxs):
    r, z, x, y1, y2, y3 = rs[ri], zs[ri], xs[ri], y1s[ri], y2s[ri], y3s[ri]
    rr, zz = np.meshgrid(r, z)
    plt.figure(figsize=(16, 4))
    plt.subplot(1, 5, 1)
    # plt.contourf(rr, zz, y1, levels=20)
    plt.scatter(rr, zz, c=y1.flatten(), marker='.')
    plt.plot(y3[:NLCFS], y3[NLCFS:], 'gray', lw=2)
    plot_vessel()
    plt.axis('equal'), plt.axis('off')
    plt.title('Y1')
    plt.colorbar()
    plt.subplot(1, 5, 2)
    # plt.contourf(rr, zz, y2, levels=20)
    plt.scatter(rr, zz, c=y2.flatten(), marker='.')
    plt.plot(y3[:NLCFS], y3[NLCFS:], 'gray', lw=2)
    plot_vessel()
    plt.axis('equal'), plt.axis('off')
    plt.title('Y2')
    plt.colorbar()
    plt.subplot(1, 5, (3,5))
    plt.bar(np.arange(x.shape[0]), x)
    plt.title('X')
    plt.tight_layout()
    plt.suptitle(f'SHOT {ri} (inputs normalized)')
    plt.show() if LOCAL else plt.savefig(f'{DS_DIR}/imgs/ds_{i}.png')
    plt.close()

In [ ]:
print('Done! Space used:')
os.system(f'du -h {TRAIN_DS_PATH}')
os.system(f'du -h {EVAL_DS_PATH}')
assert os.path.exists(TRAIN_DS_PATH), f"Dataset not saved: {TRAIN_DS_PATH}"
assert os.path.exists(EVAL_DS_PATH), f"Dataset not saved: {EVAL_DS_PATH}"
print(f"{JOBID} done")
if not LOCAL: sleep(30) # wait for files to update (for cluster)